# Spark: Getting Started
 * These instructions require a Mac with [Anaconda3](https://anaconda.com/) and [Homebrew](https://brew.sh/) installed.
 * Useful for small data only. For larger data, try [Databricks](https://databricks.com/).

## Step 0: Prerequisites & Installation

Run these commands in your terminal (just once).

```bash
# Make Homebrew aware of old versions of casks
brew tap caskroom/versions

# Install Java 1.8 (OpenJDK 8)
brew cask install adoptopenjdk8

# Install the current version of Spark
brew install apache-spark

# Install Py4J (connects PySpark to the Java Virtual Machine)
pip install py4j

# Add JAVA_HOME to .bash_profile (makes Java 1.8 your default JVM)
echo "\n# Apache Spark\nexport JAVA_HOME=$(/usr/libexec/java_home -v 1.8)" >> ~/.bash_profile

# Add SPARK_HOME to .bash_profile
echo "\nexport SPARK_HOME=/usr/local/Cellar/apache-spark/2.4.3/libexec" >> ~/.bash_profile

# Add PySpark to PYTHONPATH in .bash_profile
echo "\nexport PYTHONPATH=$SPARK_HOME/python:$PYTHONPATH" >> ~/.bash_profile

# Update current environment
source ~/.bash_profile

```

## Step 1: Create a SparkSession with a SparkContext

In [10]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [76]:
spark

In [77]:
sc

If we need a broadcast (i.e. global) variable, we can declare it like so:

In [78]:
glob = sc.broadcast(list(range(1, 3)))
glob.value

## Step 2: Download some Amazon reviews (Toys & Games)

In [14]:
# Download data (run this only once)
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz
#!gunzip reviews_Toys_and_Games_5.json.gz

## Step 3: Create a Spark DataFrame

In [15]:
df = spark.read.json('/Users/gdamico/flatiron/reviews_Toys_and_Games_5.json')

In [79]:
df.persist()

This last command, `.persist()`, simply stores the DataFrame in memory. See [this page](https://unraveldata.com/to-cache-or-not-to-cache/). It is similar to `.cache()`, but actually more flexible than the latter since you can specify which storage level you want. See [here](https://stackoverflow.com/questions/26870537/what-is-the-difference-between-cache-and-persist).

In [80]:
type(df)

In [81]:
df.show(5) # default of 20 lines

In [82]:
pdf = df.limit(5).toPandas()
pdf

In [83]:
type(pdf)

In [84]:
df.count()

In [85]:
df.columns

In [86]:
df.printSchema()

The 'nullable = true' bit means that the relevant column tolerates null values.

In [88]:
df.describe().show()

In [89]:
df.describe('overall').show()

In [29]:
reviews_df = df[['asin', 'overall']]

In [90]:
reviews_df.show()

In [31]:
def show(df, n=5):
    return df.limit(n).toPandas()

In [91]:
show(reviews_df)

In [92]:
reviews_df.count()

In [34]:
sorted_review_df = reviews_df.sort('overall')

In [93]:
show(sorted_review_df)

In [36]:
import pyspark.sql.functions as F

In [37]:
counts = reviews_df.agg(F.countDistinct('overall'))

In [94]:
counts.show()

In [39]:
query = """
SELECT overall, COUNT(*)
FROM reviews
GROUP BY overall
ORDER BY overall
"""

In [40]:
reviews_df.createOrReplaceTempView('reviews')

In [41]:
output = spark.sql(query)

In [95]:
show(output)

In [96]:
output.collect()

In [97]:
reviews_df.count() - sum(output.collect()[i][1] for i in range(5))

In [98]:
type(reviews_df)

Convert to RDD!

In [99]:
reviews_df.rdd

In [100]:
type(reviews_df.rdd)

### Count the words in the first row

In [48]:
row_one = df.first()

In [101]:
row_one

In [50]:
def word_count(text):
    return len(text.split())

In [102]:
word_count(row_one['reviewText'])

In [52]:
from pyspark.sql.types import IntegerType

#'udf' is for User Defined Function!

word_count_udf = F.udf(word_count, returnType=IntegerType())

In [53]:
review_text_col = df['reviewText']

In [54]:
counts_df = df.withColumn('wordCount', word_count_udf(review_text_col))

In [103]:
# Remember that we set the default number of lines to show at 5.

show(counts_df).T

In [104]:
from pyspark.sql.types import IntegerType
word_count_udf = F.udf(word_count, IntegerType())

# Registering our word_count() function so that we
# can use it with SQL! See documentation here:
# https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-UDFRegistration.html

df.createOrReplaceTempView('reviews')
spark.udf.register('word_count', word_count_udf)

In [57]:
query = """
SELECT asin, overall, reviewText, word_count(reviewText) AS wordCount
FROM reviews
"""

In [58]:
counts_df = spark.sql(query)

In [105]:
show(counts_df)

In [60]:
def count_all_the_things(text):
    return [len(text), len(text.split())]

In [61]:
from pyspark.sql.types import ArrayType, IntegerType
count_udf = F.udf(count_all_the_things, returnType=ArrayType(IntegerType()))

In [62]:
counts_df = df.withColumn('counts', count_udf(df['reviewText']))

In [106]:
show(counts_df, 1)

In [64]:
slim_counts_df = (
    df.drop('reviewTime')
      .drop('helpful')
      .withColumn('counts', count_udf(df['reviewText']))
      .drop('reviewText')
)

In [107]:
show(slim_counts_df, n=1)

In [108]:
aggs = counts_df.groupBy('reviewerID').agg({'overall': 'mean'})
aggs.collect()

### A few more basic commands

Please refer also to the [official programming guide](http://spark.apache.org/docs/latest/rdd-programming-guide.html).

In [67]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

In [109]:
distData

In [69]:
def multiply(a, b):
    return a * b

In [110]:
distData.reduce(multiply)

In [71]:
def subtract1(a, b):
    return a - b

In [111]:
distData.reduce(subtract1)

In [73]:
def subtract2(a, b):
    return b - a

In [112]:
distData.reduce(subtract2)

Can you explain these "subtraction" results?

In [113]:
distData.filter(lambda x: x < 4).collect()

### Reading files

```sc.textFile()``` for .txt files

`.toJSON()` for .json files

In [70]:
dfjson = counts_df.toJSON()

In [71]:
df2 = spark.read.json(dfjson)

In [114]:
df2.printSchema()

`sc.pickleFile()` for pickles

In [115]:
counts_df

In [116]:
type(df.toPandas())

In [75]:
import pickle

with open('reviews.pkl', 'wb') as frame:
    pickle.dump(df.toPandas(), frame)

In [76]:
outofthejar = sc.pickleFile('reviews.pkl')

In [117]:
type(outofthejar)